In [22]:
#There are three burst where not all detectors are on
#modify the fitting script by hand, since the data grouping will be different

def fit_script(HEDfname=None,KW1fname=None,KW2fname=None,GBMn1fname=None,GBMn2fname=None,GBMb1fname=None,outfname=None):
    
    fnames = [HEDfname,KW1fname,KW2fname,GBMn1fname,GBMn2fname,GBMb1fname,outfname]
    if all(isinstance(fname, str) for fname in fnames) == False:
        return(print("input must be string"))
    
    with open('fitting.xcm',"r") as f:
        lines = f.readlines() #198 lines of code
        for i in np.arange(0,len(lines)):
            lines[i] = lines[i].replace("HED_PHA_1.fits",HEDfname)
            lines[i] = lines[i].replace("KW201227_1_gr20.pha",KW1fname)
            lines[i] = lines[i].replace("KW201227_2_gr20.pha",KW2fname)
            lines[i] = lines[i].replace("GRB201227A_n4_gr20.pha",GBMn1fname)
            lines[i] = lines[i].replace("GRB201227A_n8_gr20.pha",GBMn2fname)
            lines[i] = lines[i].replace("GRB201227A_b0_gr20.pha",GBMb1fname)
            
    with open(outfname+".xcm","w") as out:
        for line in lines:
            out.write(line)

In [59]:
fit_script(HEDfname="bla",KW1fname="blaa",KW2fname="blaaa",GBMn1fname="la",GBMn2fname="laa",GBMb1fname="laaa",outfname="fooo")